In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
with open('key.json') as x:
    credentials = json.load(x)

In [3]:
api_key = credentials['api_key']

***Importing cleaned data from another notebook to pass to  omdb api***

In [4]:
original=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\og.csv')
reem=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\reem.csv')
three=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\three.csv')
four=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\four.csv')
five=pd.read_csv(r'C:\Users\Thund\NSS_Data_Analytics\Projects\Capstone\MovieRemakeCapstone\data\five.csv')

***Adding api friendly column***

In [5]:
original['api']=[x.replace(' ','+') for x in original['title']]
reem['api']=[x.replace(' ','+') for x in reem['title']]
three['api']=[x.replace(' ','+') for x in three['title']]
four['api']=[x.replace(' ','+') for x in four['title']]
five['api']=[x.replace(' ','+') for x in five['title']]

In [6]:
original

,Unnamed: 0,year,title,api
0,0,1963,13 Assassins,13+Assassins
1,1,1960,13 Ghosts,13+Ghosts
2,2,2005,13 Tzameti,13+Tzameti
3,3,2009,3 Idiots,3+Idiots
4,4,1957,3:10 to Yuma,3:10+to+Yuma
...,...,...,...,...
531,531,1977,The Yellow Handkerchief,The+Yellow+Handkerchief
532,532,1948,Yellow Sky,Yellow+Sky
533,533,1961,Yojimbo,Yojimbo
534,534,1968,"Yours, Mine and Ours","Yours,+Mine+and+Ours"


In [7]:
#For some reason, the year column in my three dataframe did not read in correctly. Converting it to a string for api.
three['new_year']=[str(x) for x in three['year']]
three

,Unnamed: 0,year,title,api,new_year
0,0,2017,3 Idiotas,3+Idiotas,2017
1,1,1978,The 39 Steps,The+39+Steps,1978
2,2,1985,Pyar Jhukta Nahin,Pyar+Jhukta+Nahin,1985
3,3,1999,Mann,Mann,1999
4,4,2014,Annie,Annie,2014
...,...,...,...,...,...
82,82,1957,Victor and Victoria,Victor+and+Victoria,1957
83,83,2009,Veera Madakari,Veera+Madakari,2009
84,84,2019,What Men Want,What+Men+Want,2019
85,85,2008,The Women,The+Women,2008


***testing the api***

In [8]:
endpoint = 'http://www.omdbapi.com/?apikey='+api_key+'&i=tt10028196'
response = requests.get(endpoint)

In [9]:
res=response.json()

In [10]:
res

{'Title': 'Laal Singh Chaddha',
 'Year': '2022',
 'Rated': 'PG-13',
 'Released': '11 Aug 2022',
 'Runtime': '159 min',
 'Genre': 'Comedy, Drama, Romance',
 'Director': 'Advait Chandan',
 'Writer': 'Atul Kulkarni, Winston Groom, Eric Roth',
 'Actors': 'Aamir Khan, Ahmad Ibn Umar, Kareena Kapoor',
 'Plot': 'The story of Laal Singh Chaddha, a simple man whose extraordinary journey will fill you with love, warmth and happiness.',
 'Language': 'Hindi, Punjabi, English',
 'Country': 'India',
 'Awards': '5 wins & 23 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMmQ4OTEyOTctMmFkOC00N2M4LWIyMDEtN2Y1MzQ0NzMzNDJlXkEyXkFqcGdeQXVyODE5NzE3OTE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.6/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '67%'},
  {'Source': 'Metacritic', 'Value': '46/100'}],
 'Metascore': '46',
 'imdbRating': '5.6',
 'imdbVotes': '177,595',
 'imdbID': 'tt10028196',
 'Type': 'movie',
 'DVD': 'N/A',
 'BoxOffice': '$3,401,324',
 'Pro

***Practice unpacking rating dictionary correctly***

In [11]:
rates=res['Ratings']

In [12]:
test_tom=[]

rates2=list(rates[1].values())

rates3=rates2[1]
rates3

'67%'

In [13]:
api=original['api']
year=original['year']

def api_pass(x,y):
    return 'http://www.omdbapi.com/?apikey='+api_key+'&t='+x+'&y='+y


In [14]:
api_list=api_pass(api, year)

***Creating lists of attributes***

In [15]:
title_list=[]
years=[]
age_rating=[]
release=[]
runtime=[]
genre=[]
directors=[]
box=[]
imdb=[]
rotten=[]
meta=[]

for x in api_list:
    endpoint = x  #api_pass(api, year)'http://www.omdbapi.com/?apikey='+api_key+'&t='+x+'&y='+y
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        try:
            title_list.append(res['Title'])
        except:
            title_list.append('not found')
        try:
            years.append(res['Year'])
        except:
            years.append('No Year')
        try:
            age_rating.append(res['Rated'])
        except:
            age_rating.append('Not Rated')
        try:
            release.append(res['Released'])
        except:
            release.append('No Date')
        try:
            runtime.append(res['Runtime'])
        except:
            runtime.append('No Runtime')
        try:
            genre.append(res['Genre'])
        except:
            genre.append('No Genre')
        try:
            directors.append(res['Director'])
        except:
            directors.append('Not Listed')
        try:
            box.append(res['BoxOffice'])
        except:
            box.append('No Box')
        try:
            imdb.append(res['imdbRating'])
        except:
            imdb.append('No imdb rating')
        try:
            meta.append(res['Metascore'])
        except:
            meta.append('No Metascore')
        try:
            rates=list(rates[1].values())
            if rates[0]=='Rotten Tomatoes':
                rates2=rates[1]
                rotten.append(rates2)
            else:
                continue
        except:
            rotten.append('n/a')
            

In [16]:
#original_df=pd.DataFrame(title_list)

In [17]:
original['Title']=title_list
original['Release Year']=years
original['Metacritic']=meta
original['MPAA']=age_rating
original['Release Date']=release
original['Runtime']=runtime
original['Genres']=genre
original['Directors']=directors
original['Box_Office']=box
original['IMDB']=imdb
original['Rotten_Tom']=rotten

In [18]:
original

,Unnamed: 0,year,title,api,Title,Release Year,Metacritic,MPAA,Release Date,Runtime,Genres,Directors,Box_Office,IMDB,Rotten_Tom
0,0,1963,13 Assassins,13+Assassins,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,67%
1,1,1960,13 Ghosts,13+Ghosts,13 Ghosts,1960,56,Not Rated,01 Jul 1960,85 min,"Horror, Mystery",William Castle,"$3,270,000",6.0,n/a
2,2,2005,13 Tzameti,13+Tzameti,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
3,3,2009,3 Idiots,3+Idiots,3 Idiots,2009,67,PG-13,25 Dec 2009,170 min,"Comedy, Drama",Rajkumar Hirani,"$6,532,874",8.4,n/a
4,4,1957,3:10 to Yuma,3:10+to+Yuma,3:10 to Yuma,1957,80,Not Rated,07 Aug 1957,92 min,"Drama, Thriller, Western",Delmer Daves,"$4,033,000",7.6,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,531,1977,The Yellow Handkerchief,The+Yellow+Handkerchief,The Yellow Handkerchief,1977,N/A,N/A,28 Apr 1978,109 min,"Comedy, Drama",Yôji Yamada,N/A,7.3,n/a
532,532,1948,Yellow Sky,Yellow+Sky,Yellow Sky,1948,N/A,Approved,24 Dec 1948,98 min,"Crime, Drama, Western",William A. Wellman,"$5,600,000",7.4,n/a
533,533,1961,Yojimbo,Yojimbo,Yojimbo,1961,93,Not Rated,13 Sep 1961,110 min,"Action, Drama, Thriller",Akira Kurosawa,"$46,808",8.2,n/a
534,534,1968,"Yours, Mine and Ours","Yours,+Mine+and+Ours","Yours, Mine and Ours",1968,N/A,Unrated,24 Apr 1968,111 min,"Comedy, Family",Melville Shavelson,N/A,7.1,n/a


no_title=original_df.loc[original_df[0]== 'not found']
no_title

print(original_df.shape)
print(no_title.shape)

In [19]:
api=reem['api']
year=reem['year']

api_list=api_pass(api, year)

title_list=[]
years=[]
age_rating=[]
release=[]
runtime=[]
genre=[]
directors=[]
box=[]
imdb=[]
rotten=[]
meta=[]

for x in api_list:
    endpoint = x  
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        try:
            title_list.append(res['Title'])
        except:
            title_list.append('not found')
        try:
            years.append(res['Year'])
        except:
            years.append('No Year')
        try:
            age_rating.append(res['Rated'])
        except:
            age_rating.append('Not Rated')
        try:
            release.append(res['Released'])
        except:
            release.append('No Date')
        try:
            runtime.append(res['Runtime'])
        except:
            runtime.append('No Runtime')
        try:
            genre.append(res['Genre'])
        except:
            genre.append('No Genre')
        try:
            directors.append(res['Director'])
        except:
            directors.append('Not Listed')
        try:
            box.append(res['BoxOffice'])
        except:
            box.append('No Box')
        try:
            imdb.append(res['imdbRating'])
        except:
            imdb.append('No imdb rating')
        try:
            meta.append(res['Metascore'])
        except:
            meta.append('No Metascore')
        try:
            rates=list(rates[1].values())
            if rates[0]=='Rotten Tomatoes':
                rates2=rates[1]
                rotten.append(rates2)
            else:
                continue
        except:
            rotten.append('n/a')
            


title_list

In [21]:
reem['Title']=title_list
reem['Release Year']=years
reem['Metacritic']=meta
reem['MPAA']=age_rating
reem['Release Date']=release
reem['Runtime']=runtime
reem['Genres']=genre
reem['Directors']=directors
reem['Box_Office']=box
reem['IMDB']=imdb
reem['Rotten_Tom']=rotten

In [22]:
reem

,Unnamed: 0,year,title,api,Title,Release Year,Metacritic,MPAA,Release Date,Runtime,Genres,Directors,Box_Office,IMDB,Rotten_Tom
0,0,2010,13 Assassins,13+Assassins,13 Assassins,2010,84,R,25 Sep 2010,141 min,"Action, Adventure, Drama",Takashi Miike,"$802,778",7.5,n/a
1,1,2001,Thirteen Ghosts,Thirteen+Ghosts,OCT31 - 8 - Thirteen Ghosts (2001),2001,N/A,N/A,12 Oct 2022,N/A,N/A,N/A,N/A,N/A,n/a
2,2,2010,13,13,13,2010,29,R,01 Mar 2010,91 min,"Crime, Drama, Thriller",Géla Babluani,N/A,6.0,n/a
3,3,2012,Nanban,Nanban,Nanban,2012,N/A,Not Rated,12 Jan 2012,188 min,"Comedy, Drama",S. Shankar,N/A,7.8,n/a
4,4,2007,3:10 to Yuma,3:10+to+Yuma,3:10 to Yuma,2007,76,R,07 Sep 2007,122 min,"Action, Crime, Drama",James Mangold,"$53,606,916",7.7,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,531,2008,The Yellow Handkerchief,The+Yellow+Handkerchief,The Yellow Handkerchief,2008,62,PG-13,19 Nov 2009,102 min,"Adventure, Drama, Romance",Udayan Prasad,"$318,623",6.7,n/a
532,532,1967,The Jackals,The+Jackals,The Jackals,1967,N/A,N/A,15 Nov 1967,96 min,"Adventure, Western",Robert D. Webb,N/A,5.2,n/a
533,533,1964,A Fistful of Dollars,A+Fistful+of+Dollars,A Fistful of Dollars,1964,65,R,18 Jan 1967,99 min,"Action, Drama, Western",Sergio Leone,"$14,500,000",7.9,n/a
534,534,2005,"Yours, Mine & Ours","Yours,+Mine+&+Ours","Yours, Mine & Ours",2005,38,PG,23 Nov 2005,88 min,"Comedy, Family, Romance",Raja Gosnell,"$53,412,862",5.5,n/a


In [34]:
missing_reems=reem.loc[reem['Rotten_Tom']== 'n/a']
missing_reems

,Unnamed: 0,year,title,api,Title,Release Year,Metacritic,MPAA,Release Date,Runtime,Genres,Directors,Box_Office,IMDB,Rotten_Tom
0,0,2010,13 Assassins,13+Assassins,13 Assassins,2010,84,R,25 Sep 2010,141 min,"Action, Adventure, Drama",Takashi Miike,"$802,778",7.5,n/a
1,1,2001,Thirteen Ghosts,Thirteen+Ghosts,OCT31 - 8 - Thirteen Ghosts (2001),2001,N/A,N/A,12 Oct 2022,N/A,N/A,N/A,N/A,N/A,n/a
2,2,2010,13,13,13,2010,29,R,01 Mar 2010,91 min,"Crime, Drama, Thriller",Géla Babluani,N/A,6.0,n/a
3,3,2012,Nanban,Nanban,Nanban,2012,N/A,Not Rated,12 Jan 2012,188 min,"Comedy, Drama",S. Shankar,N/A,7.8,n/a
4,4,2007,3:10 to Yuma,3:10+to+Yuma,3:10 to Yuma,2007,76,R,07 Sep 2007,122 min,"Action, Crime, Drama",James Mangold,"$53,606,916",7.7,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,531,2008,The Yellow Handkerchief,The+Yellow+Handkerchief,The Yellow Handkerchief,2008,62,PG-13,19 Nov 2009,102 min,"Adventure, Drama, Romance",Udayan Prasad,"$318,623",6.7,n/a
532,532,1967,The Jackals,The+Jackals,The Jackals,1967,N/A,N/A,15 Nov 1967,96 min,"Adventure, Western",Robert D. Webb,N/A,5.2,n/a
533,533,1964,A Fistful of Dollars,A+Fistful+of+Dollars,A Fistful of Dollars,1964,65,R,18 Jan 1967,99 min,"Action, Drama, Western",Sergio Leone,"$14,500,000",7.9,n/a
534,534,2005,"Yours, Mine & Ours","Yours,+Mine+&+Ours","Yours, Mine & Ours",2005,38,PG,23 Nov 2005,88 min,"Comedy, Family, Romance",Raja Gosnell,"$53,412,862",5.5,n/a


In [24]:
print(reem.shape)
print(missing_reems.shape)

(536, 15)
(33, 15)


In [25]:
api=three['api']
year=three['new_year']

api_list3= api_pass(api, year)

title_list=[]
years=[]
age_rating=[]
release=[]
runtime=[]
genre=[]
directors=[]
box=[]
imdb=[]
rotten=[]
meta=[]

for x in api_list3:
    endpoint = x  #api_pass(api, year)'http://www.omdbapi.com/?apikey='+api_key+'&t='+x+'&y='+y
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        try:
            title_list.append(res['Title'])
        except:
            title_list.append('not found')
        try:
            years.append(res['Year'])
        except:
            years.append('No Year')
        try:
            age_rating.append(res['Rated'])
        except:
            age_rating.append('Not Rated')
        try:
            release.append(res['Released'])
        except:
            release.append('No Date')
        try:
            runtime.append(res['Runtime'])
        except:
            runtime.append('No Runtime')
        try:
            genre.append(res['Genre'])
        except:
            genre.append('No Genre')
        try:
            directors.append(res['Director'])
        except:
            directors.append('Not Listed')
        try:
            box.append(res['BoxOffice'])
        except:
            box.append('No Box')
        try:
            imdb.append(res['imdbRating'])
        except:
            imdb.append('No imdb rating')
        try:
            meta.append(res['Metascore'])
        except:
            meta.append('No Metascore')
        try:
            rates=list(rates[1].values())
            if rates[0]=='Rotten Tomatoes':
                rates2=rates[1]
                rotten.append(rates2)
            else:
                continue
        except:
            rotten.append('n/a')
            

In [26]:
three['Title']=title_list
three['Release Year']=years
three['Metacritic']=meta
three['MPAA']=age_rating
three['Release Date']=release
three['Runtime']=runtime
three['Genres']=genre
three['Directors']=directors
three['Box_Office']=box
three['IMDB']=imdb
three['Rotten_Tom']=rotten

In [27]:
three

,Unnamed: 0,year,title,api,new_year,Title,Release Year,Metacritic,MPAA,Release Date,Runtime,Genres,Directors,Box_Office,IMDB,Rotten_Tom
0,0,2017,3 Idiotas,3+Idiotas,2017,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
1,1,1978,The 39 Steps,The+39+Steps,1978,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
2,2,1985,Pyar Jhukta Nahin,Pyar+Jhukta+Nahin,1985,Pyar Jhukta Nahin,1985,N/A,N/A,11 Jan 1985,135 min,"Family, Romance",Vijay Sadanah,N/A,6.5,n/a
3,3,1999,Mann,Mann,1999,Mann - Soul's Heart,1999,N/A,Not Rated,22 Jun 1999,160 min,"Drama, Romance",Indra Kumar,N/A,6.1,n/a
4,4,2014,Annie,Annie,2014,Annie,2014,33,PG,19 Dec 2014,118 min,"Comedy, Drama, Family",Will Gluck,"$85,911,262",5.3,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,82,1957,Victor and Victoria,Victor+and+Victoria,1957,Victor and Victoria,1957,N/A,N/A,N/A,107 min,Comedy,Karl Anton,N/A,6.2,n/a
83,83,2009,Veera Madakari,Veera+Madakari,2009,Veera Madakari,2009,N/A,Not Rated,20 Mar 2009,175 min,Action,Sudeep,N/A,6.5,n/a
84,84,2019,What Men Want,What+Men+Want,2019,What Men Want,2019,49,R,08 Feb 2019,117 min,"Comedy, Fantasy, Romance",Adam Shankman,"$54,611,903",5.4,n/a
85,85,2008,The Women,The+Women,2008,The Women,2008,27,PG-13,12 Sep 2008,114 min,"Comedy, Drama",Diane English,"$26,902,075",5.0,n/a


In [32]:
missing_threes=three.loc[reem['Title']== 'not found']
print(three.shape)
print(missing_threes.shape)

(87, 16)
(3, 16)


In [28]:
api=four['api']
year=four['year']

api_list4= api_pass(api, year)

title_list=[]
years=[]
age_rating=[]
release=[]
runtime=[]
genre=[]
directors=[]
box=[]
imdb=[]
rotten=[]
meta=[]

for x in api_list4:
    endpoint = x  #api_pass(api, year)'http://www.omdbapi.com/?apikey='+api_key+'&t='+x+'&y='+y
    response = requests.get(endpoint)
    if response.status_code != 200:
        print(f'Failed: {response.status_code}')
    else:
        res=response.json()
        try:
            title_list.append(res['Title'])
        except:
            title_list.append('not found')
        try:
            years.append(res['Year'])
        except:
            years.append('No Year')
        try:
            age_rating.append(res['Rated'])
        except:
            age_rating.append('Not Rated')
        try:
            release.append(res['Released'])
        except:
            release.append('No Date')
        try:
            runtime.append(res['Runtime'])
        except:
            runtime.append('No Runtime')
        try:
            genre.append(res['Genre'])
        except:
            genre.append('No Genre')
        try:
            directors.append(res['Director'])
        except:
            directors.append('Not Listed')
        try:
            box.append(res['BoxOffice'])
        except:
            box.append('No Box')
        try:
            imdb.append(res['imdbRating'])
        except:
            imdb.append('No imdb rating')
        try:
            meta.append(res['Metascore'])
        except:
            meta.append('No Metascore')
        try:
            rates=list(rates[1].values())
            if rates[0]=='Rotten Tomatoes':
                rates2=rates[1]
                rotten.append(rates2)
            else:
                continue
        except:
            rotten.append('n/a')
            

In [29]:
four['Title']=title_list
four['Release Year']=years
four['Metacritic']=meta
four['MPAA']=age_rating
four['Release Date']=release
four['Runtime']=runtime
four['Genres']=genre
four['Directors']=directors
four['Box_Office']=box
four['IMDB']=imdb
four['Rotten_Tom']=rotten

In [30]:
four

,Unnamed: 0,year,title,api,Title,Release Year,Metacritic,MPAA,Release Date,Runtime,Genres,Directors,Box_Office,IMDB,Rotten_Tom
0,0,2008,The 39 Steps,The+39+Steps,The 39 Steps,2008,N/A,TV-PG,28 Dec 2008,90 min,"Adventure, Crime, Mystery",James Hawes,N/A,6.3,n/a
1,1,1985,Nee Bareda Kadambari,Nee+Bareda+Kadambari,Nee Bareda Kadambari,1985,N/A,N/A,N/A,N/A,Drama,B.S. Dwarakish,N/A,N/A,n/a
2,2,1935,Brewster's Millions,Brewster's+Millions,Brewster's Millions,1935,N/A,Approved,05 Apr 1935,80 min,"Comedy, Musical",Thornton Freeland,N/A,7.3,n/a
3,3,1979,The Cat and the Canary,The+Cat+and+the+Canary,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
4,4,1998,A Perfect Murder,A+Perfect+Murder,A Perfect Murder,1998,50,R,05 Jun 1998,107 min,"Crime, Drama, Thriller",Andrew Davis,"$67,638,368",6.6,n/a
5,5,1986,Shobhraj,Shobhraj,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
6,6,2023,Dune: Part Two,Dune:+Part+Two,not found,No Year,No Metascore,Not Rated,No Date,No Runtime,No Genre,Not Listed,No Box,No imdb rating,n/a
7,7,2018,Freaky Friday,Freaky+Friday,Freaky Friday,2018,N/A,TV-G,10 Aug 2018,90 min,"Comedy, Fantasy, Musical",Steve Carr,N/A,3.9,n/a
8,8,1988,Switching Channels,Switching+Channels,Switching Channels,1988,N/A,PG,04 Mar 1988,105 min,"Comedy, Crime",Ted Kotcheff,"$9,129,999",5.9,n/a
9,9,1949,The House Across the Street,The+House+Across+the+Street,The House Across the Street,1949,N/A,Passed,10 Sep 1949,69 min,"Comedy, Crime, Drama",Richard L. Bare,N/A,6.0,n/a


In [33]:
missing_fours=four.loc[reem['Title']== 'not found']
print(four.shape)
print(missing_fours.shape)

(28, 15)
(2, 15)
